<a href="https://colab.research.google.com/github/andivaanggoro/andivaanggoromachinelearning/blob/main/DenseNET_Andiva_Kasih_Anggoro_Putra_1103204031.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Andiva Kasih Anggoro Putra

NIM : 1103204031

In [1]:
!pip install d2l

In [2]:
import torch
# library utama untuk komputasi tensor dan pembelajaran mendalam (deep learning) menggunakan PyTorch. Library ini menyediakan berbagai fungsi untuk operasi matematika dan manipulasi tensor.
from torch import nn
# digunakan untuk membangun dan melatih jaringan saraf, seperti lapisan jaringan saraf (layers), fungsi aktivasi, dan fungsi kerugian (loss functions).
import d2l.torch as d2l
# menyediakan berbagai fungsi dan utilitas yang membantu dalam pembelajaran mendalam menggunakan PyTorch. Modul ini mencakup fungsi untuk visualisasi, pemrosesan data, dan banyak lagi.


In [3]:
def conv_block(num_channels):
  # Mendefinisikan metode conv_block dengan parameter num_channels
    return nn.Sequential(
        # Mengembalikan urutan layer-layer menggunakan nn.Sequential
        nn.LazyBatchNorm2d(), nn.ReLU(),
        # Layer Batch Normalization yang akan menyesuaikan jumlah fitur secara otomatis
        nn.LazyConv2d(num_channels, kernel_size=3, padding=1))
    # Layer Convolutional yang akan menyesuaikan jumlah fitur keluaran dengan num_channels, menggunakan kernel berukuran 3x3 dan padding 1


In [4]:
class DenseBlock(nn.Module):
  # Mendefinisikan kelas DenseBlock yang merupakan subclass dari nn.Module
    def __init__(self, num_convs, num_channels):
      # Konstruktor (initializer) untuk kelas DenseBlock dengan parameter num_convs dan num_channels
        super(DenseBlock, self).__init__()
        # Memanggil konstruktor dari kelas induk (superclass) nn.Module
        layer = []
        # Inisialisasi list kosong untuk menyimpan layer
        for i in range(num_convs):
          # Iterasi sebanyak num_convs
            layer.append(conv_block(num_channels))
            # Menambahkan conv_block dengan num_channels ke dalam list layer
        self.net = nn.Sequential(*layer)
        # Menggabungkan semua layer dalam list menjadi satu urutan (sequential) dan menyimpannya dalam atribut self.net

    def forward(self, X):
      # Mendefinisikan metode forward untuk propagasi maju
        for blk in self.net:
          # Iterasi melalui setiap blok dalam self.net
            Y = blk(X)
            # Melakukan forward pass pada blok dengan input X dan menyimpan hasilnya di Y
            X = torch.cat((X, Y), dim=1)
            # Menggabungkan input X dan output Y sepanjang dimensi channel
        return X
        # Mengembalikan hasil akhir setelah semua blok dilalui


In [5]:
blk = DenseBlock(2, 10)
# Membuat instance dari DenseBlock dengan 2 conv_block dan 10 channel per block
X = torch.randn(4, 3, 8, 8)
# Menginisialisasi tensor X dengan bentuk (4, 3, 8, 8) yang berisi nilai acak dari distribusi normal
Y = blk(X)
# Melakukan forward pass dari DenseBlock pada input X dan menyimpan hasilnya di Y
Y.shape
# Mendapatkan bentuk (shape) dari tensor Y


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([4, 23, 8, 8])

In [6]:
def transition_block(num_channels):
  # Mendefinisikan fungsi transition_block dengan parameter num_channels
    return nn.Sequential(
        # Mengembalikan urutan layer-layer menggunakan nn.Sequential
        nn.LazyBatchNorm2d(), nn.ReLU(),
        # Layer Batch Normalization yang akan menyesuaikan jumlah fitur secara otomatis
        nn.ReLU(), #  Layer aktivasi ReLU
        nn.LazyConv2d(num_channels, kernel_size=1),
        # Layer Convolutional dengan kernel berukuran 1x1 dan jumlah fitur keluaran sesuai num_channels
        nn.AvgPool2d(kernel_size=2, stride=2))
        # Layer Average Pooling dengan ukuran kernel 2x2 dan stride 2



In [7]:
blk = transition_block(10)
# Membuat instance dari transition_block dengan 10 channels
blk(Y).shape
# Melakukan forward pass dari transition_block pada input Y dan mendapatkan bentuk (shape) dari outputnya


torch.Size([4, 10, 4, 4])

In [8]:
class DenseNet(d2l.Classifier):
  # Mendefinisikan kelas DenseNet yang merupakan subclass dari d2l.Classifier
    def b1(self):
      # Mendefinisikan metode b1 dalam kelas DenseNet
        return nn.Sequential(
            # Mengembalikan urutan layer-layer menggunakan nn.Sequential
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            # Layer Convolutional dengan 64 output channels, kernel berukuran 7x7, stride 2, dan padding 3
            nn.LazyBatchNorm2d(), nn.ReLU(),
            # Layer Batch Normalization yang akan menyesuaikan jumlah fitur secara otomatis
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
            # Layer MaxPooling dengan ukuran kernel 3x3, stride 2, dan padding 1


In [9]:
@d2l.add_to_class(DenseNet)
# Mendefinisikan metode atau atribut baru dalam kelas DenseNet dari modul d2l
def __init__(self, num_channels=64, growth_rate=32, arch=(4, 4, 4, 4),
             lr=0.1, num_classes=10):
   # Mendefinisikan konstruktor (initializer) untuk kelas DenseNet dengan parameter num_channels, growth_rate, arch, lr, dan num_classes
    super(DenseNet, self).__init__()
    # Memanggil konstruktor dari kelas induk (superclass) DenseNet
    self.save_hyperparameters()
    # Menyimpan hyperparameter (num_channels, growth_rate, arch, lr, num_classes) ke dalam instance
    self.net = nn.Sequential(self.b1())
    # Menginisialisasi atribut self.net sebagai objek nn.Sequential dengan blok b1 sebagai layer pertama
    for i, num_convs in enumerate(arch):
       # Iterasi melalui arsitektur yang diberikan dan indeksnya
        self.net.add_module(f'dense_blk{i+1}', DenseBlock(num_convs,
                                                          growth_rate))
        #  Menambahkan setiap DenseBlock ke dalam self.net

        # The number of output channels in the previous dense block
        num_channels += num_convs * growth_rate
        # Menambahkan jumlah channels output dari DenseBlock

        # A transition layer that halves the number of channels is added
        # between the dense blocks
        if i != len(arch) - 1:
           # Jika bukan blok terakhir, tambahkan transition_block yang mengurangi jumlah channels setengahnya
            num_channels //= 2
            # Mengurangi jumlah channels menjadi setengahnya
            self.net.add_module(f'tran_blk{i+1}', transition_block(
                num_channels))
            # Menambahkan transition_block ke dalam self.net
    self.net.add_module('last', nn.Sequential(
        # Menambahkan layer akhir ke dalam self.net
        nn.LazyBatchNorm2d(), nn.ReLU(),
        # Layer Batch Normalization yang akan menyesuaikan jumlah fitur secara otomatis
        nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
        # Layer Adaptive Average Pooling dengan output ukuran 1x1
        nn.LazyLinear(num_classes)))
         # # Layer Linear untuk klasifikasi dengan jumlah kelas sesuai num_classes
    self.net.apply(d2l.init_cnn)
    # Menerapkan fungsi inisialisasi d2l.init_cnn ke semua layer dalam self.net


In [10]:
model = DenseNet(lr=0.01)
# Membuat instance dari model DenseNet dengan learning rate (lr) sebesar 0.01
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
# Membuat instance dari Trainer dengan maksimal 10 epoch dan menggunakan 1 GPU
data = d2l.FashionMNIST(batch_size=128, resize=(96, 96))
# Memuat dataset FashionMNIST dengan ukuran batch 128 dan mengubah ukuran gambar menjadi 96x96
trainer.fit(model, data)
# Melatih model DenseNet menggunakan data FashionMNIST dengan Trainer


100%|██████████| 26421880/26421880 [00:02<00:00, 12599699.12it/s]


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 200116.20it/s]


Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3740119.96it/s]


Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6845997.78it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw




/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


KeyboardInterrupt: 